# Utiliser l'apprentissage incrémental pour former un classificateur

L'un des problèmes des bibliothèques d'apprentissage automatique traditionnelles, telles que scikit-learn, est qu'elles offrent rarement la possibilité d'entraîner des modèles sur de gros volumes de données, ce qui, par coïncidence, est le meilleur type de données pour les réseaux de neurones profonds. À quoi bon avoir de grandes quantités de données si nous ne pouvons pas les utiliser ?

Heureusement, il existe un moyen de contourner cette limitation, et cela s'appelle l'apprentissage incrémental. Dans cette recette, nous utiliserons une puissante bibliothèque, creme, pour entraîner un classificateur sur un ensemble de données trop volumineux pour être stocké en mémoire.

nous utiliserons creme, une bibliothèque expérimentale spécialement conçue pour entraîner des modèles d'apprentissage automatique sur d'énormes ensembles de données trop volumineux pour être stockés en mémoire. Pour installer la crème, exécutez la commande suivante :.

In [15]:
!pip install creme==0.5.1

     |████████████████████████████████| 1.6 MB 7.4 MB/s 
  Created wheel for mmh3: filename=mmh3-2.5.1-cp37-cp37m-linux_x86_64.whl size=39689 sha256=8a6de954546bcaa8a9a0afa71505cde4f9c283f87df7310f244e8877c61caa98
  Stored in directory: /root/.cache/pip/wheels/ae/45/25/90e097a519143b2dca74cd93a056894a965f27908103e01799
Successfully built mmh3


In [13]:
Nous utiliserons l'ensemble de données features.hdf5 que nous avons généré dans Implémentation d'un extracteur de caractéristiques à l'aide d'une recette de réseau pré-entraînée dans ce chapitre, qui contient des informations codées sur les images pivotées à partir de l'ensemble de données Stanford Cars. Nous supposons que l'ensemble de données se trouve à l'emplacement suivant : //content/sample_data/features.hdf5

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [4]:
!kaggle datasets download -d jessicali9530/stanford-cars-dataset

100% 1.82G/1.82G [00:21<00:00, 82.6MB/s]
100% 1.82G/1.82G [00:21<00:00, 90.0MB/s]


In [14]:
#!unzip stanford-cars-dataset.zip

Nous stockerons les caractéristiques extraites au format HDF5, un protocole hiérarchique binaire conçu pour stocker de très grands ensembles de données numériques sur disque, tout en conservant la facilité d'accès et de calcul au niveau des lignes. Vous pouvez en savoir plus sur HDF5 ici : https://portal.hdfgroup.org/display/HDF5/HDF5.

Suivez ces étapes pour terminer cette recette :

**1.** Importez tous les packages nécessaires :

In [16]:
import pathlib

import h5py
from creme import stream
from creme.linear_model import LogisticRegression
from creme.metrics import Accuracy
from creme.multiclass import OneVsRestClassifier
from creme.preprocessing import StandardScaler

**2.** Déﬁnissez une fonction qui enregistrera un ensemble de données sous forme de chier CSV :

In [21]:
def write_dataset(output_path, feats, labels, batch_size):
    feature_size = feats.shape[1]
    csv_columns = ['class'] + [f'feature_{i}'
                               for i in range(feature_size)]
    #Nous aurons une colonne pour la classe de chaque caractéristique et autant de colonnes 
    #"d'éléments dans chaque vecteur de caractéristique. Ensuite, écrivons le contenu du fichier CSV par lots, en commençant par l'en-tête
    dataset_size = labels.shape[0]
    with open(output_path, 'w') as f:
        f.write(f'{",".join(csv_columns)}\n')
        #Extraire le lot dans cette itération
        for batch_number, index in \
                enumerate(range(0, dataset_size, batch_size)):
            print(f'Processing batch {batch_number + 1} of '
                  f'{int(dataset_size / float(batch_size))}')

            batch_feats = feats[index: index + batch_size]
            batch_labels = labels[index: index + batch_size]
            # Maintenant, écrivez toutes les lignes du lot
            for label, vector in \
                    zip(batch_labels, batch_feats):
                vector = ','.join([str(v) for v in vector])
                f.write(f'{label},{vector}\n')

**3.**Charger le jeu de données au format HDF5

In [18]:
dataset_path = str(pathlib.Path('/content/car_ims_rotated') / 'features.hdf5')
db = h5py.File(dataset_path, 'r')

**4.** Définir l'index de division pour séparer les données en morceaux d'entraînement (80 %) et de test (20 %)

In [19]:
TRAIN_PROPORTION = 0.8
SPLIT_INDEX = int(db['labels'].shape[0] * TRAIN_PROPORTION)

**5.**Écrire les sous-ensembles d'entraînement et de test sur le disque en tant que fichiers CSV

In [22]:
BATCH_SIZE = 256
write_dataset('train.csv',
              db['features'][:SPLIT_INDEX],
              db['labels'][:SPLIT_INDEX],
              BATCH_SIZE)
write_dataset('test.csv',
              db['features'][SPLIT_INDEX:],
              db['labels'][SPLIT_INDEX:],
              BATCH_SIZE)

Processing batch 1 of 25
Processing batch 2 of 25
Processing batch 3 of 25
Processing batch 4 of 25
Processing batch 5 of 25
Processing batch 6 of 25
Processing batch 7 of 25
Processing batch 8 of 25
Processing batch 9 of 25
Processing batch 10 of 25
Processing batch 11 of 25
Processing batch 12 of 25
Processing batch 13 of 25
Processing batch 14 of 25
Processing batch 15 of 25
Processing batch 16 of 25
Processing batch 17 of 25
Processing batch 18 of 25
Processing batch 19 of 25
Processing batch 20 of 25
Processing batch 21 of 25
Processing batch 22 of 25
Processing batch 23 of 25
Processing batch 24 of 25
Processing batch 25 of 25
Processing batch 26 of 25
Processing batch 1 of 6
Processing batch 2 of 6
Processing batch 3 of 6
Processing batch 4 of 6
Processing batch 5 of 6
Processing batch 6 of 6
Processing batch 7 of 6


**6.**creme nous oblige à spécifier le type de chaque colonne du fichier CSV en tant que dict. exemple, le bloc suivant spécifie que la classe doit être codée en tant qu'entier, tandis que les colonnes restantes, correspondant aux fonctionnalités, doivent être de type flottant

In [23]:
FEATURE_SIZE = db['features'].shape[1]
types = {f'feature_{i}': float for i in range(FEATURE_SIZE)}
types['class'] = int

**7.** Dans le code suivant, nous définissons un pipeline de crème, où chaque entrée sera standardisée avant d'être transmise au classificateur. Parce qu'il s'agit d'un problème multi-classes, nous devons envelopper LogisticRegression avec OneVsRestClassifier

In [24]:
model = StandardScaler()
model |= OneVsRestClassifier(LogisticRegression())

**8.** Définir la précision comme métrique cible et créer un itérateur sur l'ensemble de données train.csv

In [25]:
metric = Accuracy()
dataset = stream.iter_csv('train.csv',
                          target_name='class',
                          converters=types)

**9.** Former le classificateur, un exemple à la fois. Imprimez la précision de fonctionnement tous les 100 exemples

In [26]:
print('Training started...')
for i, (X, y) in enumerate(dataset):
    predictions = model.predict_one(X)
    model = model.fit_one(X, y)
    metric = metric.update(y, predictions)

    if i % 100 == 0:
        print(f'Update {i} - {metric}')

print(f'Final - {metric}')

Training started...
Update 0 - Accuracy: 0.00%
Update 100 - Accuracy: 82.18%
Update 200 - Accuracy: 87.06%
Update 300 - Accuracy: 90.03%
Update 400 - Accuracy: 91.52%
Update 500 - Accuracy: 92.61%
Update 600 - Accuracy: 93.18%
Update 700 - Accuracy: 94.01%
Update 800 - Accuracy: 94.51%
Update 900 - Accuracy: 94.89%
Update 1000 - Accuracy: 94.91%
Update 1100 - Accuracy: 95.10%
Update 1200 - Accuracy: 95.34%
Update 1300 - Accuracy: 95.54%
Update 1400 - Accuracy: 95.57%
Update 1500 - Accuracy: 95.87%
Update 1600 - Accuracy: 96.06%
Update 1700 - Accuracy: 96.18%
Update 1800 - Accuracy: 96.28%
Update 1900 - Accuracy: 96.11%
Update 2000 - Accuracy: 96.30%
Update 2100 - Accuracy: 96.43%
Update 2200 - Accuracy: 96.55%
Update 2300 - Accuracy: 96.52%
Update 2400 - Accuracy: 96.67%
Update 2500 - Accuracy: 96.68%
Update 2600 - Accuracy: 96.77%
Update 2700 - Accuracy: 96.74%
Update 2800 - Accuracy: 96.79%
Update 2900 - Accuracy: 96.76%
Update 3000 - Accuracy: 96.87%
Update 3100 - Accuracy: 96.90%
U

**10.**Créez un itérateur sur le fichier test.csv

In [27]:
metric = Accuracy()
test_dataset = stream.iter_csv('test.csv',
                               target_name='class',
                               converters=types)

**11.** Évaluez le modèle sur l'ensemble de test une fois de plus, un échantillon à la fois

In [28]:
print('Testing model...')
for i, (X, y) in enumerate(test_dataset):
    predictions = model.predict_one(X)
    metric = metric.update(y, predictions)

    if i % 1000 == 0:
        print(f'(TEST) Update {i} - {metric}')

print(f'(TEST) Final - {metric}')

Testing model...
(TEST) Update 0 - Accuracy: 100.00%
(TEST) Update 1000 - Accuracy: 98.20%
(TEST) Final - Accuracy: 98.65%


Après quelques minutes, nous devrions avoir un modèle avec une précision d'environ 99% sur l'ensemble de test. Nous verrons cela plus en détail dans la section suivante.

Souvent, même si nous disposons d'énormes quantités de données, nous ne pouvons pas toutes les utiliser en raison de limitations matérielles ou logicielles (dans la recette Formation d'un classificateur simple sur les fonctionnalités extraites, nous avons dû utiliser 50%, car nous ne pouvions pas tout garder en mémoire). Cependant, avec l'apprentissage incrémentiel (également connu sous le nom d'apprentissage en ligne), nous pouvons former des modèles d'apprentissage automatique traditionnels par lots, de la même manière que nous pouvons le faire avec les réseaux de neurones.

Dans cette recette, afin de saisir la totalité du vecteur de caractéristiques de notre jeu de données Stanford Cars, nous avons dû écrire à la fois les jeux d'entraînement et de test dans des fichiers CSV. Ensuite, nous avons formé LogisticRegression et l'avons enveloppé dans OneVsRestClassifier, qui a appris à détecter les degrés de rotation dans les vecteurs de caractéristiques des images. Enfin, nous avons atteint une précision très satisfaisante de 99% sur l'ensemble de test